In [ ]:
import json
import pandas as pd
from arcgis.gis import GIS
from arcgis.features import GeoAccessor, GeoSeriesAccessor

In [ ]:
agol = GIS(profile='adoezema_AGO')
print(agol.users.me.username)

In [ ]:
field_data = agol.content.get(itemid='3b9b6fda7d864e3bb804a7bbcd82f702')

Functions

In [ ]:
def get_unsound_records(lyr):
    #TODO: Query by todays date
    keep_fields = ('GlobalID', 'FACILITYID', 'CIPMOID', 'CONDITION', 'OHMComment', 'OHM_Status',  'last_edited_date', 'last_edited_user','SHAPE')
    lyr.query(where="CONDITION<>'Sound'", out_fields=keep_fields)
    return lyr.query(where="CONDITION<>'Sound'", out_fields=keep_fields)

In [ ]:
    def transform_attachment_df(attachment_df):
        url_df = pd.DataFrame(attachment_df.groupby('PARENTGLOBALID')['IMAGE_PREVIEW'].
                              apply(lambda x: ', '.join(x).split(', ')))
        photo_series = url_df.IMAGE_PREVIEW.apply(lambda x: len(x))
        photo_field_names = {i: f'Photo {i+1}' for i in range(photo_series.max())}
        url_df = pd.DataFrame(url_df.IMAGE_PREVIEW.apply(pd.Series))
        url_df.reset_index(inplace=True)
        url_df.rename(columns=photo_field_names, inplace=True)
        return url_df

Main flow control

In [ ]:
from IPython.core.display import HTML

In [ ]:

for lyr in field_data.layers:
    lyr_name = lyr.properties.name
    if lyr_name not in ('Miscellaneous Point', 'Neighborhoods'):
        data = get_unsound_records(lyr).sdf
        data.insert(loc=0, column='Layer Name', value=lyr_name)
        data['OHM_Status'].replace({'1': 'Incomplete', '2': 'Surveyed', '3': 'Attemped', '4': 'Delivered'}, inplace=True)
        #TODO: Move Photo processing to function
        photo_collection = []
        for index, row in data.iterrows():
            photo = lyr.attachments.search(global_ids=row['GlobalID'], attachment_types=['image/jpeg','image/jpg'])
            if photo:
                photo = transform_attachment_df(pd.DataFrame(photo)).to_dict('records')[0]
                photo_collection.append(photo)
        photo_df = pd.DataFrame(photo_collection)
        result = pd.merge(data, photo_df, how='outer', left_on=['GlobalID'], right_on=['PARENTGLOBALID'])
        result.drop('SHAPE', axis=1, inplace=True)
    #TODO Combine individual Layers into one dataframe

In [ ]:
HTML(result.to_html(escape=False))